####**Model Training**

In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve,confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

warnings.filterwarnings("ignore")
%matplotlib inline

In [8]:
#reading the dataset

df = pd.read_csv("/content/breast_cancer.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [9]:
#droping id and Unnamed: 32 columns

df = df.drop(['id','Unnamed: 32'], axis=1)
df.head(2)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [10]:
df.shape

(569, 31)

**Encoading Categorical column**

In [11]:
df=pd.get_dummies(data=df, drop_first=True)
df.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis_M
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


###Preparing X and Y variables

In [12]:
X = df.drop('diagnosis_M', axis=1)

In [13]:
X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   radius_mean              569 non-null    float64
 1   texture_mean             569 non-null    float64
 2   perimeter_mean           569 non-null    float64
 3   area_mean                569 non-null    float64
 4   smoothness_mean          569 non-null    float64
 5   compactness_mean         569 non-null    float64
 6   concavity_mean           569 non-null    float64
 7   concave points_mean      569 non-null    float64
 8   symmetry_mean            569 non-null    float64
 9   fractal_dimension_mean   569 non-null    float64
 10  radius_se                569 non-null    float64
 11  texture_se               569 non-null    float64
 12  perimeter_se             569 non-null    float64
 13  area_se                  569 non-null    float64
 14  smoothness_se            5

In [15]:
y = df['diagnosis_M']
y

0      1
1      1
2      1
3      1
4      1
      ..
564    1
565    1
566    1
567    1
568    0
Name: diagnosis_M, Length: 569, dtype: uint8

In [16]:
num_features = X.select_dtypes(exclude='object').columns

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("StandardScaler",numeric_transformer, num_features)
    ]
)

In [17]:
X = preprocessor.fit_transform(X)

In [18]:
X.shape

(569, 30)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((455, 30), (114, 30))

###**Create an Evaluate Function to give all metrics after Training**

In [20]:
def evaluate_model(true,predicted):

  '''
    This function takes in true values and predicted values
    Returns: Accuracy, F1-Score, Precision, Recall, Roc-auc Score
  '''
  acc = accuracy_score(true, predicted) # Calculate Accuracy
  f1 = f1_score(true, predicted) # Calculate F1-score
  precision = precision_score(true, predicted) # Calculate Precision
  recall = recall_score(true, predicted)  # Calculate Recall
  roc_auc = roc_auc_score(true, predicted) #Calculate Roc
  return acc, f1 , precision, recall, roc_auc

In [21]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest' : RandomForestClassifier(),
    'Decision Tree' : DecisionTreeClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'K-Neighbors Classifier': KNeighborsClassifier(),
    'XGB Classifier': XGBClassifier(),
    'Cat Boost Classifier': CatBoostClassifier(verbose=False),
    'Ada Boost Classifier': AdaBoostClassifier()
}

model_list = []
acc = []
f1 = []
precision = []
recall = []
roc_auc = []


for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(X_train, y_train) #Train Model

  #make prediction
  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)

  #Evaluate Train and test dataset
  model_train_acc, model_train_f1, model_train_precision, model_train_recall, model_train_roc_auc = evaluate_model(y_train, y_train_pred)
  
  model_test_acc, model_test_f1, model_test_precision, model_test_recall, model_test_roc_auc = evaluate_model(y_test, y_test_pred)


  print(list(models.keys())[i])
  model_list.append(list(models.keys())[i])

  print('Model performance for Training set')
  print("- Accuracy : {:.4f}".format(model_train_acc))
  print("- f1 score : {:.4f}".format(model_train_f1))
  print("- precision : {:.4f}".format(model_train_precision))
  print("- recall : {:.4f}".format(model_train_recall))
  print("- roc_auc : {:.4f}".format(model_train_roc_auc))

  print("-------------------------------------------------------")

  print('Model performance for Test set')
  print("- Accuracy : {:.4f}".format(model_test_acc))
  print("- f1 score : {:.4f}".format(model_test_f1))
  print("- precision : {:.4f}".format(model_test_precision))
  print("- recall : {:.4f}".format(model_test_recall))
  print("- roc_auc : {:.4f}".format(model_test_roc_auc))

  acc.append(model_test_acc)
  f1.append(model_test_f1)
  precision.append(model_test_precision)
  recall.append(model_test_recall)
  roc_auc.append(model_test_roc_auc)

  print("="*40)
  print('\n')



Logistic Regression
Model performance for Training set
- Accuracy : 0.9868
- f1 score : 0.9821
- precision : 0.9880
- recall : 0.9763
- roc_auc : 0.9847
-------------------------------------------------------
Model performance for Test set
- Accuracy : 0.9737
- f1 score : 0.9647
- precision : 0.9762
- recall : 0.9535
- roc_auc : 0.9697


Random Forest
Model performance for Training set
- Accuracy : 1.0000
- f1 score : 1.0000
- precision : 1.0000
- recall : 1.0000
- roc_auc : 1.0000
-------------------------------------------------------
Model performance for Test set
- Accuracy : 0.9649
- f1 score : 0.9524
- precision : 0.9756
- recall : 0.9302
- roc_auc : 0.9581


Decision Tree
Model performance for Training set
- Accuracy : 1.0000
- f1 score : 1.0000
- precision : 1.0000
- recall : 1.0000
- roc_auc : 1.0000
-------------------------------------------------------
Model performance for Test set
- Accuracy : 0.9474
- f1 score : 0.9302
- precision : 0.9302
- recall : 0.9302
- roc_auc : 0

###**Result**

In [22]:
pd.DataFrame(list(zip(model_list, acc, f1, precision, recall, roc_auc)), 
             columns = ['Model Name','Accuracy','F1 score','Precision','Recall','roc_auc']).sort_values(by=['Accuracy'])

,Model Name,Accuracy,F1 score,Precision,Recall,roc_auc
2,Decision Tree,0.947368,0.930233,0.930233,0.930233,0.943990
4,K-Neighbors Classifier,0.947368,0.930233,0.930233,0.930233,0.943990
3,Gradient Boosting,0.956140,0.941176,0.952381,0.930233,0.951032
5,XGB Classifier,0.956140,0.941176,0.952381,0.930233,0.951032
1,Random Forest,0.964912,0.952381,0.975610,0.930233,0.958074
0,Logistic Regression,0.973684,0.964706,0.976190,0.953488,0.969702
6,Cat Boost Classifier,0.973684,0.964706,0.976190,0.953488,0.969702
7,Ada Boost Classifier,0.973684,0.964706,0.976190,0.953488,0.969702


###**Logistic Regression**

In [23]:
Logit_model = LogisticRegression(fit_intercept=True)
Logit_model = Logit_model.fit(X_train, y_train)
y_pred = Logit_model.predict(X_test)
score = accuracy_score(y_test, y_pred)*100
print("Accuracy of the model is %.2f" %score)

Accuracy of the model is 97.37


###**Confusion Metrix**

In [24]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[70  1]
 [ 2 41]]


In [ ]:
from sklearn.metrics import plot_confusion_matrix

#plots Confusion matrix
plot_confusion_matrix(Logit_model, X_test, y_test, cmap='Blues', values_format='d')